In [ ]:
!pip install gurobipy

In [ ]:
import numpy as np
import gurobipy as gp

In [ ]:
M = 10000.0
N = 5
distMat = np.array([[M,132,217,164,58],[132,M,290,201,79],[217,290,M,113,303],[164,201,113,M,196],[58,79,303,196,M]])



In [ ]:
mod = gp.Model()
x = mod.addMVar((N,N),vtype='B')
u = mod.addMVar(N)

In [ ]:
mod.setObjective(gp.quicksum(x[i,j]*distMat[i,j] for i in range(N) for j in range(N)))

In [ ]:

# leave each city once
leave = mod.addConstrs((gp.quicksum(x[i,j] for i in range(N))==1) for j in range(N))
# enter each city once
enter = mod.addConstrs((gp.quicksum(x[i,j] for j in range(N))==1) for i in range(N))
# u constraints
ucons = mod.addConstrs((u[i] - u[j] + N*x[i,j] <= (N-1)) for i in range(1,N) for j in range(1,N) if i != j)

In [ ]:
len(ucons)

In [ ]:
(N-1)**2 - (N-1)

In [ ]:
mod.Params.OutputFlag = 0 # tell gurobi to shut up!!
mod.optimize()

In [ ]:
mod.objVal

In [ ]:
where = 0
miles_traveled = 0
print('Start in city 0.')
for city in range(N):
    where_new = np.where(x.x[where,:])[0][0]
    dist = distMat[where,where_new]
    miles_traveled += dist
    print('Then go to city '+str(where_new)+', traveling '+str(dist)+' miles along the way.')
    where = where_new
print('For a total of '+str(miles_traveled)+' miles.')

In [ ]:
# this path may not be unique, there are other paths that are optimal
# but they'll all have the same total miles traveled!
# for example 0 -> 2 -> 3 -> 1 -> 4 -> 0
# also takes 668 miles!

In [ ]:
seq = [0, 2, 3, 1, 4, 0]
miles_traveled2 = 0
for loc in range(N):
    where = seq[loc]
    where_new = seq[loc+1]
    dist = distMat[where,where_new]
    miles_traveled2 += dist
print(miles_traveled2)

In [ ]:
# I have no idea how many other paths are optimal...